In [1]:
%pip install gdown

train = '1-8HVDyZkJwLxn4Dc-SqNWlZZkd3jFvF_'
val = '1-7saiHjQojSkvxbrkz1d4ebryH252Iox'
test = '1-D-vwSDK6qPH2TN8owr1Uy3JQSKitpVH'

train_embeddings_per_word = '1kc018iaKNu8Y1DJd4WfyxWOxTQNNYt-h'
val_embeddings_per_word = '1--fg57qeqXEFrLqYvoVcN_9-o-Pzc26I'
test_embeddings_per_word = '1--K6JKVyn8UE9oo-4wzf5B6IzJ0JrI7c'

train_embeddings_per_sentence = '1-5dkTRFtcYbJJf5AWufBxgMba8FRSN7i'
val_embeddings_per_sentence = '1-FwXE-za1j3cBk9Cvw5T3DnzkJAGiY-M'
test_embeddings_per_sentence = '1-8icZrYP0oNUTnh6mwSGwK9fLAS1gcQU'

uncleaned_train_embedding = '13UONDopwe65QnuMK6jJWQUIdyx3OI-FA'
uncleaned_val_embedding = '1-0eCfZ8xSvLAI4RuC2e1WLf_Z09GmAaN'
uncleaned_test_embedding = '1-0CfuTbiTOIfDr4R76MO4Hd7GVeFlSrn'

cleaned_train_embedding = '1-4ss3i6KEvWWuFegO95aB3itKopXwM3T'
cleaned_val_embedding = '1-5vw77DSI1gsMdnquVXkZrA9xai57f4o'
cleaned_test_embedding = '1ItgcQIno9rkbyIuDv3xtU6PRjURBIgos'

model_path = '1V9PmKPW7kLwpHGNk6xhEKlB2QWQ3BTNG'

!gdown {train}
!gdown {val}
!gdown {test}

!gdown {cleaned_train_embedding}
!gdown {cleaned_val_embedding}
!gdown {cleaned_test_embedding}

Note: you may need to restart the kernel to use updated packages.
Downloading...
From: https://drive.google.com/uc?id=1-8HVDyZkJwLxn4Dc-SqNWlZZkd3jFvF_
To: /kaggle/working/train.pkl
100%|███████████████████████████████████████| 3.00M/3.00M [00:00<00:00, 172MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-7saiHjQojSkvxbrkz1d4ebryH252Iox
To: /kaggle/working/val.pkl
100%|████████████████████████████████████████| 643k/643k [00:00<00:00, 61.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-D-vwSDK6qPH2TN8owr1Uy3JQSKitpVH
To: /kaggle/working/test.pkl
100%|█████████████████████████████████████████| 638k/638k [00:00<00:00, 102MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-4ss3i6KEvWWuFegO95aB3itKopXwM3T
From (redirected): https://drive.google.com/uc?id=1-4ss3i6KEvWWuFegO95aB3itKopXwM3T&confirm=t&uuid=33663f32-d4db-44ec-ad0b-a16c4af4c0f6
To: /kaggle/working/ls_train_embeddings.pkl
100%|███████████████████████████████████████| 2.16G/2.16G [00:10<00:0

In [2]:
import pandas as pd
import numpy as np
import pickle
import torch
import shap
import seaborn as sns
import tensorflow as tf
import keras
import keras_tuner
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from lime.lime_text import LimeTextExplainer
from transformers import BertModel, AutoTokenizer, TFAutoModel
from time import time

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/o

In [3]:
classes = ['none', 'anger', 'joy', 'sadness', 'love', 'sympathy', 'surprise', 'fear']

with open('/kaggle/working/train.pkl', 'rb') as f:
    train = pickle.load(f)
    
with open('/kaggle/working/val.pkl', 'rb') as f:
    val = pickle.load(f)

with open('/kaggle/working/test.pkl', 'rb') as f:
    test = pickle.load(f)

with open('/kaggle/working/ls_train_embeddings.pkl', 'rb') as f:
    train_embeddings = pickle.load(f)
    
with open('/kaggle/working/ls_val_embeddings.pkl', 'rb') as f:
    val_embeddings = pickle.load(f)
    
with open('/kaggle/working/ls_test_embeddings.pkl', 'rb') as f:
    test_embeddings = pickle.load(f)

In [4]:
encoder = OneHotEncoder()

y_train = encoder.fit_transform(train['label'].values.reshape(-1,1)).toarray()
y_val = encoder.transform(val['label'].values.reshape(-1,1)).toarray()
y_test = encoder.transform(test['label'].values.reshape(-1,1)).toarray()

In [5]:
max_length = 50

In [6]:
X_train = train_embeddings
X_val = val_embeddings
X_test = test_embeddings

In [7]:
BATCH_SIZE = 128
EPOCHS = 50
EMBED_SIZE = 300
LEARNING_RATE =  0.001

early_stopping_monitor = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True)

In [8]:
def build_gru(hp):
    model = Sequential()
    gru_units = hp.Int('gru_base_units', min_value=32, max_value=256, step=2, sampling='log')
    gru_layers = hp.Int('num_gru_layers', 0, 5)
    gru_dropout = hp.Float('gru_dropout', min_value=0, max_value=0.5)
    model.add(tf.keras.layers.GRU(gru_units*(2**gru_layers), return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(gru_dropout))
    for i in range(gru_layers): 
        model.add(tf.keras.layers.GRU(gru_units*(2**(gru_layers-i)), return_sequences=(i < gru_layers - 1)))
        model.add(Dropout(gru_dropout))

    for i in range(hp.Int('num_dense_layers', 0, 3)):
        model.add(Dense(units=gru_units/(2**i)))
        model.add(Dropout(hp.Float('dense_dropout', min_value=0, max_value=0.5)))   

    model.add(Dense(8, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2)),
                loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [9]:
def build_gru_2(hp):
    model = Sequential()
    model.add(tf.keras.layers.GRU(hp.Int('gru_units', min_value=32, max_value=1024, step=2, sampling='log'), return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(hp.Float('dense_dropout', min_value=0, max_value=0.5)))
    model.add(Dense(8, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2)),
                loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [10]:
def build_gru_3(hp):
    model = Sequential()
    gru_units = hp.Int('gru_base_units', min_value=32, max_value=256, step=2, sampling='log')
    gru_dropout = hp.Float('dropout', min_value=0, max_value=0.5)
    model.add(GRU(4*gru_units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(gru_dropout))
    
    model.add(GRU(2*gru_units, return_sequences=True))
    model.add(Dropout(gru_dropout))
    
    model.add(GRU(gru_units, return_sequences=False))
    model.add(Dropout(gru_dropout))
    
    model.add(Dense(8, activation='softmax'))

    model.compile(optimizer=Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2)),
                loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [16]:
class Custom_Bayesian(keras_tuner.tuners.GridSearch):
  def run_trial(self, trial, *args, **kwargs):
    kwargs['batch_size'] = int(trial.hyperparameters.Int('batch_size', 16, 512, step=2, sampling='log')*1)
    return super(Custom_Bayesian, self).run_trial(trial, *args, **kwargs)


In [ ]:
tuner = Custom_Bayesian(
    build_gru,
    objective='val_accuracy',
    max_trials=20,
    overwrite=True,
    directory='my_dir2',
    project_name='gru')

In [ ]:
tuner.search(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10), keras.callbacks.TensorBoard("tb_logs")])

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
tuner2 = Custom_Bayesian(
    build_gru_2,
    objective='val_accuracy',
    max_trials=30,
    overwrite=True,
    directory='my_dir2',
    project_name='gru')

In [ ]:
tuner2.search(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10), keras.callbacks.TensorBoard("tb_logs")])

In [ ]:
tuner2.get_best_hyperparameters()[0].values

In [17]:
tuner3 = Custom_Bayesian(
    build_gru_3,
    objective='val_accuracy',
    max_trials=100,
    overwrite=True,
    directory='my_dir3',
    project_name='gru')

In [18]:
tuner3.search(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10), keras.callbacks.TensorBoard("tb_logs")])

Trial 100 Complete [00h 00m 48s]
val_accuracy: 0.7165563106536865

Best val_accuracy So Far: 0.734437108039856
Total elapsed time: 02h 17m 19s


In [19]:
tuner3.get_best_hyperparameters()[0].values

{'gru_base_units': 32,
 'dropout': 0.0,
 'learning_rate': 0.006535000000000002,
 'batch_size': 64.0}

In [20]:
%load_ext tensorboard

In [21]:
%tensorboard --logdir tb_logs

In [22]:
!cd /kaggle/working

In [23]:
!ls

ls_test_embeddings.pkl	 ls_val_embeddings.pkl	tb_logs   train.pkl
ls_train_embeddings.pkl  my_dir3		test.pkl  val.pkl


In [24]:
!zip -r tb_logs.zip tb_logs

  adding: tb_logs/ (stored 0%)
  adding: tb_logs/012/ (stored 0%)
  adding: tb_logs/012/execution0/ (stored 0%)
  adding: tb_logs/012/execution0/validation/ (stored 0%)
  adding: tb_logs/012/execution0/validation/events.out.tfevents.1692815429.8b84f6f74c69.28.50.v2 (deflated 76%)
  adding: tb_logs/012/execution0/events.out.tfevents.1692815407.8b84f6f74c69.28.48.v2 (deflated 23%)
  adding: tb_logs/012/execution0/train/ (stored 0%)
  adding: tb_logs/012/execution0/train/events.out.tfevents.1692815417.8b84f6f74c69.28.49.v2 (deflated 88%)
  adding: tb_logs/019/ (stored 0%)
  adding: tb_logs/019/execution0/ (stored 0%)
  adding: tb_logs/019/execution0/validation/ (stored 0%)
  adding: tb_logs/019/execution0/validation/events.out.tfevents.1692815985.8b84f6f74c69.28.71.v2 (deflated 76%)
  adding: tb_logs/019/execution0/train/ (stored 0%)
  adding: tb_logs/019/execution0/train/events.out.tfevents.1692815976.8b84f6f74c69.28.70.v2 (deflated 88%)
  adding: tb_logs/019/execution0/events.out.tfeven

In [25]:
!cd ../..

In [26]:
!ls

ls_test_embeddings.pkl	 ls_val_embeddings.pkl	tb_logs      test.pkl	val.pkl
ls_train_embeddings.pkl  my_dir3		tb_logs.zip  train.pkl
